# Simple RAG App Tutorial

## Overview

This tutorial shows how to build a Retrieval Augmented Generation (RAG) application using LangChain. RAG combines document retrieval with language model generation to answer questions about specific data sources.

## Setup

### Installation

In [ ]:
!pip install langchain-text-splitters langchain-community langgraph langchain-openai langchain-core bs4

### Environment Variables

In [ ]:
import getpass
import os

# Set up OpenAI API key for embeddings
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API key: ")

# Set up Google API key for LLM
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter Google API key: ")

## Core Components

### 1. Initialize Models and Vector Store

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# Initialize chat model
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

# Initialize embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Initialize vector store
vector_store = InMemoryVectorStore(embeddings)

### 2. Load and Process Documents

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load documents from web
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200
)
all_splits = text_splitter.split_documents(docs)

# Add documents to vector store
_ = vector_store.add_documents(documents=all_splits)

print(f"Loaded {len(docs)} documents")
print(f"Split into {len(all_splits)} chunks")

### 3. Create RAG Application with LangGraph

In [ ]:
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load RAG prompt
prompt = hub.pull("rlm/rag-prompt")

# Define application state
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Define retrieval function
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

# Define generation function
def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

# Build and compile the graph
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

print("RAG application created successfully!")

## Usage Examples

### Basic Query

In [ ]:
# Ask a question
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

### Stream Responses

In [ ]:
# Stream the steps
for step in graph.stream(
    {"question": "What is Task Decomposition?"}, 
    stream_mode="updates"
):
    print(f"{step}\n")

### Stream Tokens

In [ ]:
# Stream individual tokens
for message, metadata in graph.stream(
    {"question": "What is Task Decomposition?"}, 
    stream_mode="messages"
):
    print(message.content, end="")

## Key Concepts

- **Document Loading**: Use WebBaseLoader to extract content from web pages
- **Text Splitting**: Break large documents into smaller, searchable chunks
- **Embeddings**: Convert text to vector representations for similarity search
- **Vector Store**: Store and retrieve document embeddings
- **RAG Pipeline**: Combine retrieval and generation for informed responses
- **LangGraph**: Orchestrate the RAG workflow with streaming support

## Next Steps

- Add conversation memory for chat-like interactions
- Implement query analysis for better retrieval
- Add metadata filtering for more precise results
- Deploy using LangGraph Platform